In [15]:
## Imports

import pandas as pd
import re

In [ ]:
"""
Take the downloaded Excel. Read Columns Tech_ID and References

"""


df = pd.read_excel("reFuel_TechDatabase.xlsx", sheet_name="ConvTech", header = 2, usecols="F,AW")
df.head()

,ehubX Tech ID,References
0,T_C_Dummy_Elec_ElDrs_El13,NaN
1,T_C_Dummy_Elec_El13_ElExp,NaN
2,T_C_Dummy_Thermal_ThBdg_ThSh,NaN
3,T_C_Dummy_Thermal_ThBdg_ThDhw,NaN
4,T_C_Dummy_Thermal_ThDh_ThHt,NaN


In [32]:
"""
Define which parameters require a reference in the references column

"""

def parse_references(cell: str) -> dict:
    if not isinstance(cell,str) or not cell.strip():
        return {}
    pairs = {}

    for chunk in [c for c in cell.split(';') if c.strip()]:
        m = re.search(r'\s*"?(?P<params>[^":;]+)"?\s*:\s*(?P<ref>.+)\s*$',chunk)
        if m:
            param = m.group('params').strip().strip(',')
            ref = m.group('ref').strip().strip(',;')
            if param:
                pairs[param] =ref
    return pairs

def classify_references(df: pd.DataFrame,
                         references_col: str = "References",
                         id_col: str | None = "ID",
                         required_params: list[str] = None) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    if required_params is None:
        required_params = ["Economic Lifetime","CAPEX Power Capacity","OPEX Power Capacity"]
    req_set = set(required_params)
    
    work = df.copy()

    parsed = work[references_col].apply(parse_references)

    present_sets = parsed.apply(lambda d: set(d.keys()))
    missing_sets = present_sets.apply(lambda s:list(req_set-s))
    present_counts = present_sets.apply(lambda s: len(req_set & s))

    has_all_flag = present_sets.apply(lambda s: "all" in [p.lower() for p in s])

    work["_parsed_refs"] = parsed
    work["_missing_params"] = missing_sets
    work["_present_count"] = present_counts
    work["_is_empty"] = parsed.apply(lambda d: len(d) ==0)
    work["_is_complete"] = (missing_sets.apply(lambda miss: len(miss) ==0)) | has_all_flag

    if id_col is None or id_col not in work.columns:
        work["_RowID"] = work.index
        id_col = "_RowID"
        
    empty_refs = work[work["_is_empty"]][[id_col, references_col]].rename(columns={id_col: "RowID_or_ID"})
    partial_refs = work[(~work["_is_empty"]) & (~work["_is_complete"])][
        [id_col, references_col, "_parsed_refs", "_missing_params", "_present_count"]
    ].rename(columns={id_col: "RowID_or_ID",
                      "_parsed_refs": "ParsedPairs",
                      "_missing_params": "MissingParams",
                      "_present_count": "NumPresent"})
    complete_refs = work[work["_is_complete"]][
        [id_col, references_col, "_parsed_refs"]
    ].rename(columns={id_col: "RowID_or_ID",
                      "_parsed_refs": "ParsedPairs"})

    return empty_refs.reset_index(drop=True), partial_refs.reset_index(drop=True), complete_refs.reset_index(drop=True)
                                            

In [33]:
# Example usage:
df = pd.read_excel("reFuel_TechDatabase.xlsx", sheet_name="ConvTech", header=2)
required = ["Economic Lifetime","CAPEX Power Capacity","OPEX Power Capacity"]  # <-- your required parameters
empty_df, partial_df, complete_df = classify_references(df = df,
                                                       references_col="References",
                                                       id_col=None,        # or None to use row index
                                                       required_params=required)
print("Empty:\n", empty_df.head())
print("Partial:\n", partial_df.head())
print("Complete:\n", complete_df.head())

Empty:
    RowID_or_ID References
0            0        NaN
1            1        NaN
2            2        NaN
3            3        NaN
4            4        NaN
Partial:
    RowID_or_ID                                         References  \
0           22   "Capex Power Capacity", "Opex Power Capacity"...   
1           23   "Capex Power Capacity", "Opex Power Capacity"...   
2           24   "Capex Power Capacity", "Opex Power Capacity"...   
3           25   "Capex Power Capacity", "Opex Power Capacity"...   
4           26   "Capex Power Capacity", "Opex Power Capacity"...   

                                         ParsedPairs  \
0  {'Opex Power Capacity': 'report_VSE_2025', 'Ec...   
1  {'Opex Power Capacity': 'report_VSE_2025', 'Ec...   
2  {'Opex Power Capacity': 'report_VSE_2025', 'Ec...   
3  {'Opex Power Capacity': 'report_VSE_2025', 'Ec...   
4  {'Opex Power Capacity': 'report_VSE_2025', 'Ec...   

                                 MissingParams  NumPresent  
0  [CAPEX Pow